# **콘텐츠 검색 및 DB 입력 자동화**

### 1. 드라마 검색
- 방영중/한국 드라마

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re

# Chrome 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

# 네이버 검색 페이지 URL
base_url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=드라마"

# 페이지 이동을 위한 초기 설정
driver.get(base_url)
time.sleep(2)

# 결과 저장 리스트
drama_list = []

while True:
    # 드라마 정보 크롤링
    drama_elements = driver.find_elements(By.CSS_SELECTOR, "li.info_box")

    for drama in drama_elements:
        try:
            # 이미지 추출
            try:
                image_element = drama.find_element(By.CSS_SELECTOR, "div.thumb_area a.thumb img")  # <img> 태그 선택
                src = image_element.get_attribute("src")  # 이미지 URL 가져오기
            except NoSuchElementException:
                src = None  # 이미지가 없으면 None 처리

            # 제목과 href에서 os 값 추출
            title_element = drama.find_element(By.CSS_SELECTOR, "strong.title a")
            keyword = title_element.text.strip()
            href = title_element.get_attribute("href")

            # href에서 os 값 추출
            os_value = None
            if "os=" in href:
                os_value = href.split("os=")[-1].split("&")[0]

            # 방송사 및 방영 시간 정보 처리
            broadcaster = drama.find_element(By.CSS_SELECTOR, "div.main_info a.broadcaster").text.strip()
            raw_text = drama.find_element(By.CSS_SELECTOR, "div.main_info").text
            # 방송사 부분 제거
            broadcast_info = raw_text.replace(broadcaster, "").strip()
            # 괄호만 제거
            broadcast_info = re.sub(r"[()]", "", broadcast_info).strip()

            drama_list.append({
                "keyword": keyword,
                "os": os_value,
                "naver_image": src,
                "Broadcasting Station": broadcaster,
                "Broadcast Date": broadcast_info
            })
        except Exception as e:
            print("Error:", e)

    # 페이지네이션 확인
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "a.pg_next")
        if "on" in next_button.get_attribute("class"):  # 다음 버튼 활성화 여부 확인
            next_button.click()
            time.sleep(2)
        else:
            break  # 다음 버튼이 비활성화면 종료
    except:
        break

# 크롤링 결과를 DataFrame으로 변환
df = pd.DataFrame(drama_list)

# keyword 값이 없는 행 삭제 (NaN + 빈 문자열 제거)
df = df[df["keyword"].str.strip() != ""].reset_index(drop=True)

# 드라이버 종료
driver.quit()

# 데이터 정리
import re

df['os_source'] = 'NAVER'

# combined_keyword 컬럼 추가
df["naver_name"] = df['keyword']
df["combined_keyword"] = df['keyword']

# 'naver_info' 컬럼 추가 (딕셔너리 형태의 문자열)
df['naver_info'] = df.apply(lambda row:
    f"{{'Broadcasting Station': '{row['Broadcasting Station']}', 'Broadcast Date': '{row['Broadcast Date']}'}}",
    axis=1
)

# 컬럼 순서 정리
desired_order = ['os', 'os_source', 'keyword', "naver_name", 'combined_keyword', 'naver_info', 'naver_image']
df = df[desired_order]

df

,os,os_source,keyword,naver_name,combined_keyword,naver_info,naver_image
0,34412615,NAVER,그놈은 흑염룡,그놈은 흑염룡,그놈은 흑염룡,"{'Broadcasting Station': 'tvN', 'Broadcast Dat...",https://search.pstatic.net/common?type=o&size=...
1,36245101,NAVER,킥킥킥킥,킥킥킥킥,킥킥킥킥,"{'Broadcasting Station': 'KBS2', 'Broadcast Da...",https://search.pstatic.net/common?type=o&size=...
2,34320158,NAVER,언더커버 하이스쿨,언더커버 하이스쿨,언더커버 하이스쿨,"{'Broadcasting Station': 'MBC', 'Broadcast Dat...",https://search.pstatic.net/common?type=o&size=...
3,36278783,NAVER,보물섬,보물섬,보물섬,"{'Broadcasting Station': 'SBS', 'Broadcast Dat...",https://search.pstatic.net/common?type=o&size=...
4,35442173,NAVER,독수리 5형제를 부탁해!,독수리 5형제를 부탁해!,독수리 5형제를 부탁해!,"{'Broadcasting Station': 'KBS2', 'Broadcast Da...",https://search.pstatic.net/common?type=o&size=...
5,35458934,NAVER,마녀,마녀,마녀,"{'Broadcasting Station': '채널A', 'Broadcast Dat...",https://search.pstatic.net/common?type=o&size=...
6,35068298,NAVER,친절한 선주씨,친절한 선주씨,친절한 선주씨,"{'Broadcasting Station': 'MBC', 'Broadcast Dat...",https://search.pstatic.net/common?type=o&size=...
7,35463505,NAVER,신데렐라 게임,신데렐라 게임,신데렐라 게임,"{'Broadcasting Station': 'KBS2', 'Broadcast Da...",https://search.pstatic.net/common?type=o&size=...
8,35133419,NAVER,결혼하자 맹꽁아!,결혼하자 맹꽁아!,결혼하자 맹꽁아!,"{'Broadcasting Station': 'KBS1', 'Broadcast Da...",https://search.pstatic.net/common?type=o&size=...


### 2. 배우 검색
- 방영중 드라마 등장인물(출연진)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
from bs4 import BeautifulSoup

# Chrome 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 안 함

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)


def get_actors_from_drama(df):
    """ 드라마 리스트를 순회하며 배우 정보를 크롤링하는 함수 (Selenium + BeautifulSoup) """
    actor_list = []

    for _, row in df.iterrows():
        keyword, os_value = row["keyword"], row["os"]
        cast_url = f"https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjkw&pkid=57&os={os_value}&qvt=0&query={keyword}%20등장인물"

        driver.get(cast_url)
        time.sleep(1)  # 페이지 로딩 대기

        soup = BeautifulSoup(driver.page_source, "html.parser")  # Selenium 결과를 파싱

        # 배우 목록 크롤링
        actor_elements = soup.select("div.title_box span.sub_text a._text")
        for actor in actor_elements:
            actor_name = actor.text.strip()
            actor_href = actor["href"]
            actor_os = actor_href.split("os=")[-1].split("&")[0] if "os=" in actor_href else None
            actor_list.append({"keyword": actor_name, "os": actor_os})

    return pd.DataFrame(actor_list)

def get_actor_profiles(df):
    """ 배우 리스트를 순회하며 프로필 정보를 크롤링하는 함수 (Selenium + BeautifulSoup) """
    profile_list = []

    for _, row in df.iterrows():
        actor_name, os_value = row["keyword"], row["os"]
        profile_url = f"https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&os={os_value}&qvt=0&query={actor_name}%20프로필"

        driver.get(profile_url)
        time.sleep(1)  # 페이지 로딩 대기

        soup = BeautifulSoup(driver.page_source, "html.parser")  # Selenium 결과를 파싱

        profile_data = {"keyword": actor_name, "os": os_value}

        # 네이버 이름 크롤링
        naver_name_tag = soup.select_one("span.area_text_title strong._text")
        profile_data["naver_name"] = naver_name_tag.text.strip() if naver_name_tag else None

        # 기본 정보 크롤링
        for group in soup.select("div.detail_info div.info_group"):
            label_tag = group.select_one("dt")
            value_tag = group.select_one("dd")
            if label_tag and value_tag:
                label, value = label_tag.text.strip(), value_tag.text.strip()
                profile_data[label] = value

        # 프로필 이미지 크롤링
        # image_tag = soup.select_one("a.thumb._item img._img") or soup.select_one("div.img_scroll ul.img_list li._item:first-child img")
        image_tag = (
            soup.select_one("a.thumb._item img._img") or  # 기존 방식
            soup.select_one("div.img_scroll ul.img_list li._item:first-child img") or  # 스크롤 이미지
            soup.select_one("a.thumb img._img")  # 숫자가 있는 이미지
        )
        profile_data["naver_image"] = image_tag["src"] if image_tag else None

        # 사이트 정보 크롤링
        site_dt = soup.select_one("dt:-soup-contains('사이트')")  # "사이트"라는 텍스트 포함한 dt 태그 찾기
        if site_dt:
            site_links = site_dt.find_next_sibling("dd").select("a")
            for link in site_links:
                profile_data[link.text.strip()] = link["href"]

        profile_list.append(profile_data)

    return pd.DataFrame(profile_list)


def clean_profile_data(df):
    """ 크롤링한 배우 프로필 데이터를 정리하는 함수 """
    df["os_source"] = "NAVER"

    # 출생일 변환 함수
    def format_birthdate(birth_text):
        match = re.search(r"(\d{4})\.(\d{2})\.(\d{2})", str(birth_text))
        return f"{match.group(1)}-{match.group(2)}-{match.group(3)}" if match else None

    df["출생"] = df["출생"].apply(format_birthdate)

    # 결측값을 빈 문자열("")로 변환 후 strip 적용
    df["keyword"] = df["keyword"].fillna("").astype(str).str.strip()
    df["naver_name"] = df["naver_name"].fillna("").astype(str).str.strip()
    df["다른이름"] = df["다른이름"].fillna("").astype(str).str.strip()

    # combined_keyword 생성 (빈값 제외 후 중복 제거)
    df["combined_keyword"] = df.apply(lambda row: ", ".join(
        {row["keyword"], row["naver_name"], row["다른이름"]} - {""}
    ), axis=1)

    # naver_info 생성 (빈 값 제외)
    df["naver_info"] = df.apply(lambda row: str({
        col: row[col] for col in df.columns
        if col not in ["keyword", "os", "naver_name", "naver_image", "os_source", "combined_keyword"]
        and pd.notna(row[col]) and str(row[col]).strip()  # NaN, 빈 문자열("") 제외
    }), axis=1)

    # 컬럼 정리
    return df[["os", "os_source", "keyword", "naver_name", "combined_keyword", "naver_info", "naver_image"]]


# 크롤링 실행
actor_df = get_actors_from_drama(df)
profile_df = get_actor_profiles(actor_df)
profile_df = clean_profile_data(profile_df)

# 드라이버 종료
driver.quit()

# 엑셀 파일로 저장
import os
from datetime import datetime

# 🔹 오늘 날짜 가져오기 (YYYYMMDD 형식)
today_date = datetime.today().strftime('%Y%m%d')

# 🔹 저장할 디렉토리 경로
save_dir = "저장 경로"

# 🔹 디렉토리가 없으면 생성
os.makedirs(save_dir, exist_ok=True)

# 🔹 파일 경로 설정 (날짜 포함)
file_path = os.path.join(save_dir, f"{today_date}_dramaactor.xlsx")

# 🔹 DataFrame을 Excel 파일로 저장
profile_df.to_excel(file_path, index=False)

# 결과 확인
profile_df

In [ ]:
profile_df

,os,os_source,keyword,naver_name,combined_keyword,naver_info,naver_image
0,150316,NAVER,문가영,문가영,문가영,"{'출생': '1996-07-10', '나이': '28세', '소속사': '피크제이...",https://search.pstatic.net/common?type=b&size=...
1,13932584,NAVER,최현욱,최현욱,"최현욱, Choi Hyun Wook","{'출생': '2002-01-30', '나이': '23세', '소속사': 'GOLD...",https://search.pstatic.net/common?type=b&size=...
2,109564,NAVER,임세미,임세미,"임세미, Se Mi Lim","{'소속사': '눈컴퍼니', '사이트': '공식홈페이지, 인스타그램, 유튜브', '...",https://search.pstatic.net/common?type=b&size=...
3,160436,NAVER,곽시양,곽시양,곽시양,"{'출생': '1987-01-15', '나이': '38세', '소속사': '드로잉엔...",https://search.pstatic.net/common?type=b&size=...
4,99762,NAVER,반효정,반효정,반효정,"{'출생': '1942-11-27', '나이': '82세', '소속사': '기억컴퍼...",https://search.pstatic.net/common?type=b&size=...
...,...,...,...,...,...,...,...
131,101967,NAVER,전수경,전수경,전수경,"{'출생': '1966-07-12', '나이': '58세', '소속사': '애닉이엔...",https://search.pstatic.net/common?type=b&size=...
132,98056,NAVER,차광수,차광수,차광수,"{'출생': '1966-02-15', '나이': '59세', '데뷔': '1991 ...",https://search.pstatic.net/common?type=b&size=...
133,98956,NAVER,이칸희,이칸희,"Lee Khan Hee, 이칸희","{'소속사': '스케치이앤엠', '신체': '166cm, 50kg, AB형', '가...",https://search.pstatic.net/common?type=b&size=...
134,97761,NAVER,정의갑,정의갑,"정의갑, Jung Eui Gap","{'소속사': '매니지먼트 율', '신체': '185cm, 80kg, O형', '가...",https://search.pstatic.net/common?type=b&size=...


### 3. 성별 분류
- 딥페이스 모델 사용
- 추가 검증 필요

In [ ]:
import os
import pandas as pd
from deepface import DeepFace

# 입력 파일 경로
input_file_path = "input_file_path"
output_file_path = "output_file_path"

# 이미지 폴더 경로
image_folder = "저장 경로"

# 데이터프레임 로드
df = pd.read_excel(input_file_path)

# 성별 분석 및 추가
def analyze_gender(image_id):
    try:
        image_path = os.path.join(image_folder, f"{image_id}.jpg")

        if not os.path.isfile(image_path):
            print(f"Image file not found: {image_path}")
            return "Image Not Found"

        # DeepFace로 성별 분석
        analysis = DeepFace.analyze(img_path=image_path, actions=['gender'])

        if isinstance(analysis, list):
            analysis = analysis[0]

        return analysis['dominant_gender']
    except Exception as e:
        print(f"Error analyzing image {image_id}: {e}")
        return "Error"

# '성별' 열 추가
df['성별'] = df['os'].apply(lambda x: analyze_gender(x))

# 결과 저장
df.to_excel(output_file_path, index=False)
print(f"성별 정보가 추가된 데이터가 저장되었습니다. 저장 경로: {output_file_path}")

In [ ]:
import pandas as pd

final_df = pd.read_excel('저장 경로', index_col=False) # 136 rows × 8 columns

# '성별' 컬럼에서 "Error" 또는 "Image Not Found" 개수 확인
error_count = final_df[final_df["성별"] == "Error"].shape[0]
not_found_count = final_df[final_df["성별"] == "Image Not Found"].shape[0]

# 결과 출력
print(f"🚨 분석 중 오류(Error): {error_count}개")
print(f"⚠ 이미지 파일 없음(Image Not Found): {not_found_count}개")
print(f"총 문제 발생 건수: {error_count + not_found_count}개")

# "Image Not Found"가 아닌 행만 유지
final_df = final_df[final_df["성별"] != "Image Not Found"]

#
final_df = final_df.drop(columns = 'Unnamed: 7') # 126 rows × 8 columns

# 'os' 컬럼 기준으로 중복 제거 (첫 번째 값만 유지)
final_df = final_df.drop_duplicates(subset=["os"], keep="first") # 125 rows × 8 columns

# 인덱스 리셋 (필요한 경우)
final_df = final_df.reset_index(drop=True)

final_df

🚨 분석 중 오류(Error): 0개
⚠ 이미지 파일 없음(Image Not Found): 10개
총 문제 발생 건수: 10개


,os,os_source,keyword,naver_name,combined_keyword,naver_info,naver_image,성별
0,150316,NAVER,문가영,문가영,문가영,"{'출생': '1996-07-10', '나이': '28세', '소속사': '피크제이...",https://search.pstatic.net/common?type=b&size=...,Woman
1,13932584,NAVER,최현욱,최현욱,"Choi Hyun Wook, 최현욱","{'출생': '2002-01-30', '나이': '23세', '소속사': 'GOLD...",https://search.pstatic.net/common?type=b&size=...,Man
2,109564,NAVER,임세미,임세미,"임세미, Se Mi Lim","{'소속사': '눈컴퍼니', '사이트': '공식홈페이지, 인스타그램, 유튜브', '...",https://search.pstatic.net/common?type=b&size=...,Woman
3,160436,NAVER,곽시양,곽시양,곽시양,"{'출생': '1987-01-15', '나이': '38세', '소속사': '드로잉엔...",https://search.pstatic.net/common?type=b&size=...,Man
4,6202612,NAVER,김영아,김영아,"김영아, Kim Young Ah","{'신체': '170cm', '사이트': '인스타그램', '인스타그램': 'http...",https://search.pstatic.net/common?type=b&size=...,Woman
...,...,...,...,...,...,...,...,...
120,106161,NAVER,최수린,최수린,최수린,"{'소속사': '심스토리(주)', '신체': '169cm', '가족': '언니\xa...",https://search.pstatic.net/common?type=b&size=...,Woman
121,101967,NAVER,전수경,전수경,전수경,"{'출생': '1966-07-12', '나이': '58세', '소속사': '애닉이엔...",https://search.pstatic.net/common?type=b&size=...,Woman
122,98056,NAVER,차광수,차광수,차광수,"{'출생': '1966-02-15', '나이': '59세', '데뷔': '1991 ...",https://search.pstatic.net/common?type=b&size=...,Man
123,97761,NAVER,정의갑,정의갑,"Jung Eui Gap, 정의갑","{'소속사': '매니지먼트 율', '신체': '185cm, 80kg, O형', '가...",https://search.pstatic.net/common?type=b&size=...,Man


### 4. 데베 추가
- 기존에 존재하는 키워드는 category만 추가

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import urllib.parse

# 🔹 MySQL 연결 정보
db_config = {
    "dialect": "mysql",
    "driver": "pymysql",
    "username": "username",
    "password": urllib.parse.quote("password"),  # '@' 처리
    "server": "server",
    "port": 3306,
    "database": "database",
}

# 🔹 SQLAlchemy 연결 URL 생성
DB_URL = "{dialect}+{driver}://{username}:{password}@{server}:{port}/{database}".format(**db_config)

# 🔹 DB 연결
engine = create_engine(DB_URL)

# 🔹 SQL 쿼리
insert_keyword_query = text("""
insert_keyword_query
""")

insert_middle_query = text("""
insert_middle_query
""")

# 🔹 기존 데이터에서 찾는 쿼리
select_existing_keyword_query = text("""
select_existing_keyword_query
""")

# 🔹 두 개의 DataFrame을 하나로 합치기
merged_df = pd.concat([df, final_df], ignore_index=True)

# 🔹 여러 개의 데이터를 한 번에 삽입
with engine.begin() as conn:
    for row in merged_df.to_dict(orient="records"):

        # 🔹 gender 값에 따라 category_id 설정
        gender = row.get("성별", None)  # gender 값 가져오기
        category_id = category_id  # 기본값 설정 (NaN이거나 빈 값일 경우)

        if pd.notna(gender):  # NaN이 아닌 경우 처리
            gender = str(gender).strip().lower()  # 문자열 변환 후 정리
            if gender == "woman":
                category_id = category_id
            elif gender == "man":
                category_id = category_id

        # 🔹 기존 데이터 확인
        keyword_id = None
        if # os 값이 존재하는 경우
            keyword_id = existing_keyword_id  # ✅ 기존 데이터 사용

        if keyword_id is None:  # 기존 데이터가 없으면 새로 삽입
            result = conn.execute(insert_keyword_query, row)
            keyword_id = conn.execute(text("SELECT LAST_INSERT_ID();")).scalar()

        # 🔹 keyword_id가 존재하는 경우
        if keyword_id is not None:
            conn.execute(insert_middle_query, {"keyword_id": keyword_id, "category_id": category_id})

print("데이터 삽입 완료!")


데이터 삽입 완료!
